In [29]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_anthropic import ChatAnthropic
from langchain_core.pydantic_v1 import BaseModel, Field

from rich.console import Console
from rich.markdown import Markdown

console = Console()


class BusinessConcept(BaseModel):
    headline: str = Field(description="Business or product concept headline")
    description: str = Field(description="Brief description of the concept")
    target_audience: str = Field(description="Target audience for the concept")
    pricing: str = Field(description="How to price the business or product")
    marketing: str = Field(description="How to market the business or product")
    stand_out: str = Field(description="How to make the business or product stand out from competitors")
    dos: str = Field(description="List of dos for the concept")
    donts: str = Field(description="List of don'ts for the concept")

class BusinessPlan(BaseModel):
    milestone_plan: str = Field(description="Milestone plan for the business or product")
    gant_chart: str = Field(description="Gant chart for the business or product")
    raid_chart: str = Field(description="Raid chart for the business or product")
    task_table: str = Field(description="Task table for the business or product")
    

gpt4 = ChatOpenAI(model="gpt-4o")
claud3 = ChatAnthropic(model='claude-3-opus-20240229')

first_prompt = ChatPromptTemplate.from_template(
    "Develop a business or product concept and around the following topic: {topic}. List dos and don'ts for the concept. Include a brief description of the concept and the target audience, how to price and market the business or product, and how to make it stand out from competitors. The output should be markdown."
)
second_prompt = ChatPromptTemplate.from_template(
    "Create a milestone plan, a gant chart, a raid chart and a task table. based on the following markdown document: {markdown} "
)

concept_parser = JsonOutputParser(pydantic_object=BusinessConcept)
plan_parser = JsonOutputParser(pydantic_object=BusinessPlan)

concept_prompt = PromptTemplate(
    #template="Answer the user query.\n{format_instructions}\n{query}\n",
    template="Develop a business or product concept and around the following topic: {topic}. List dos and don'ts for the concept. Include a brief description of the concept and the target audience, how to price and market the business or product, and how to make it stand out from competitors. \n{format_instructions}. \nInside the json the output should be markdown.",
    input_variables=["topic"],
    partial_variables={"format_instructions": concept_parser.get_format_instructions()},
)

plan_prompt = PromptTemplate(
    #template="Answer the user query.\n{format_instructions}\n{query}\n",
    template="Create a milestone plan, a gant chart, a raid chart and a task table. based on the following json/markdown document: {markdown} \n{format_instructions}. \nInside the json the output should be markdown, without any ```markdown annotations.",
    input_variables=["markdown"],
    partial_variables={"format_instructions": plan_parser.get_format_instructions()},
)

#first_chain =  first_prompt | gpt4 | StrOutputParser()
first_chain =  concept_prompt | gpt4 | concept_parser

#second_chain = second_prompt | gpt4 | StrOutputParser()
second_chain = plan_prompt | gpt4 | plan_parser

complete_chain = ({
    "topic": itemgetter("topic"),
    "markdown": first_chain
    }
    | RunnablePassthrough.assign(plans=second_chain)
    | RunnablePassthrough.assign(markdown=first_chain)
)

In [31]:
test = complete_chain.invoke({"topic":"cyber security bike rides"})

OutputParserException: Invalid json output: Here is the output formatted as a JSON instance that conforms to the provided schema:

{
  "milestone_plan": "Milestone 1: Planning (2 months)\n- Develop detailed tour routes and educational content\n- Hire and train guides\n- Secure bikes, helmets, and other gear\n- Establish partnerships with local organizations\n- Finalize pricing and booking system\n\nMilestone 2: Launch (1 month)\n- Conduct initial marketing push\n- Run first tours and gather feedback\n- Refine tour format and content based on feedback\n- Expand marketing efforts\n\nMilestone 3: Growth (6 months)\n- Continue running and refining tours\n- Expand to new cities/regions\n- Develop additional tour formats (e.g., weekend trips)\n- Pursue larger partnerships and sponsorships\n\nMilestone 4: Expansion (1 year+)\n- Franchise model or expansion to international markets\n- Develop online cyber security courses\n- Expand product line (e.g., CyberCycle branded gear)\n- Continual impr

In [23]:
from rich import print
print(test['markdown']['headline'])
print(test['markdown']['description'])
print(test['markdown']['target_audience'])
print(test['markdown']['pricing'])
print(test['markdown']['marketing'])
print(test['markdown']['stand_out'])
print(test['markdown']['dos'])
print(test['markdown']['donts'])

print(test['plans']['milestone_plan'])
print(test['plans']['gant_chart'])
print(test['plans']['raid_chart'])
print(test['plans']['task_table'])

InnovateHub: The Ultimate Ideation Platform

InnovateHub is a cutting-edge ideation platform designed to facilitate brainstorming, idea generation, and 
collaborative innovation for businesses, startups, and creative teams. It integrates AI-driven insights, real-time 
collaboration tools, and customizable templates to streamline the ideation process.

Businesses, startups, creative teams, innovation managers, and product development teams.

InnovateHub offers a tiered pricing model: 
- Free Basic Plan: Includes limited access to basic features and templates. 
- Pro Plan ($19.99/month): Offers advanced features, AI insights, and increased storage. 
- Enterprise Plan (custom pricing): Tailored solutions with full customization, dedicated support, and advanced 
analytics.

To market InnovateHub, focus on digital marketing strategies including SEO optimization, content marketing (blogs, 
whitepapers, case studies), social media advertising, and partnerships with industry influencers. Additionally, 
offer free trials and webinars to showcase the platform's capabilities.

...

- Don't overload the platform with unnecessary features.
- Don't ignore user feedback and requests for improvements.
- Don't compromise on data security and user privacy.
- Don't neglect the importance of marketing and brand presence.
- Don't underestimate the value of offering free trials and demos.

#### Milestone Plan

1. **Project Initiation**
   - Define project scope and objectives
   - Identify stakeholders
   - Develop project charter
   - Timeline: Month 1

2. **Platform Development**
   - Design user interface and experience
   - Develop core features and functionalities
   - Integrate with popular tools like Slack and Trello
   - Timeline: Months 2-5

3. **Beta Testing**
   - Conduct internal testing
   - Invite beta users and gather feedback
   - Refine platform based on feedback
   - Timeline: Months 6-7

4. **Marketing Launch**
   - Implement social media campaigns
   - Launch content marketing initiatives
   - Partner with innovation consultants
   - Timeline: Month 8

5. **Official Launch**
   - Release Free, Pro, and Enterprise tiers
   - Offer free webinars and trials
   - Leverage success stories and testimonials
   - Timeline: Month 9

6. **Post-Launch Support and Maintenance**
   - Provide robust customer support
   - Gather continuous user feedback
   - Regular updates and maintenance
   - Ongoing

#### Gant Chart

| Task                        | Start Date  | End Date    | Duration | Responsible  |
|-----------------------------|-------------|-------------|----------|--------------|
| Project Initiation          | 2023-01-01  | 2023-01-31  | 1 month  | Project Lead |
| Platform Development        | 2023-02-01  | 2023-05-31  | 4 months | Dev Team     |
| Beta Testing                | 2023-06-01  | 2023-07-31  | 2 months | QA Team      |
| Marketing Launch            | 2023-08-01  | 2023-08-31  | 1 month  | Marketing    |
| Official Launch             | 2023-09-01  | 2023-09-30  | 1 month  | All Teams    |
| Post-Launch Support & Maintenance | 2023-10-01 | Ongoing | Ongoing  | Support Team |

#### Raid Chart

| Risk                       | Impact       | Mitigation Strategy                            |
|----------------------------|--------------|-----------------------------------------------|
| Delayed Development        | High         | Allocate buffer time and prioritize features   |
| User Adoption Challenges   | Medium       | Offer free trials and strong marketing efforts |
| Data Security Issues       | High         | Implement robust security protocols            |
| Integration Failures       | Medium       | Conduct thorough testing before launch         |

| Assumption                 | Impact       | Validation Strategy                            |
|----------------------------|--------------|-----------------------------------------------|
| High demand for innovation platform | High | Market research and surveys                    |
| Integration will be seamless         | Medium | Beta testing with diverse user groups         |

| Issue                      | Impact       | Resolution Strategy                            |
|----------------------------|--------------|-----------------------------------------------|
| Bug reports from beta users           | High | Prompt bug fixing and updates                 |

| Dependency                 | Impact       | Management Strategy                            |
|----------------------------|--------------|-----------------------------------------------|
| Integration with third-party tools    | High | Close coordination with tool providers        |

#### Task Table

| Task                                   | Description                                           | Due Date    | 
Assignee     |
|----------------------------------------|-------------------------------------------------------|-------------|---
-----------|
| Define project scope and objectives    | Outline the goals and deliverables for InnovateCo     | 2023-01-07  | 
Project Lead |
| Design user interface and experience   | Create wireframes and design the platform UI/UX       | 2023-02-28  | 
Design Team  |
| Develop core features                  | Implement main functionalities like brainstorming tools| 2023-04-30  | 
Dev Team     |
| Integrate with Slack and Trello        | Ensure seamless integration with popular tools        | 2023-05-31  | 
Dev Team     |
| Conduct internal testing               | Test the platform internally to identify any issues   | 2023-06-15  | QA
Team      |
| Gather feedback from beta users        | Collect and analyze feedback from beta testing phase  | 2023-07-31  | QA
Team      |
| Launch marketing campaigns             | Start social media and content marketing campaigns    | 2023-08-01  | 
Marketing    |
| Release Free, Pro, and Enterprise tiers| Make different pricing tiers available to users       | 2023-09-01  | 
Sales        |
| Provide customer support               | Set up support channels and address user issues       | 2023-10-01  | 
Support Team |
| Gather user feedback                   | Continuously collect user feedback for improvements   | Ongoing     | 
All Teams    |

In [ ]:
from rich import print
print(test['markdown']['headline'])
print(test['markdown']['description'])
print(test['markdown']['target_audience'])
print(test['markdown']['pricing'])
print(test['markdown']['marketing'])
print(test['markdown']['stand_out'])
print(test['markdown']['dos'])
print(test['markdown']['donts'])

print(test['plans']['milestone_plan'])
print(test['plans']['gant_chart'])
print(test['plans']['raid_chart'])
print(test['plans']['task_table'])

In [28]:
gpt_md = Markdown(test['plans']['task_table'])
console.print(gpt_md)

Task Table                                                     

                                                                                                                   
  Task                                      Description                                 Due Date     Assignee      
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Define project scope and objectives       Outline the goals and deliverables for      2023-01-07   Project Lead  
                                            InnovateCo                                                             
  Design user interface and experience      Create wireframes and design the platform   2023-02-28   Design Team   
                                            UI/UX                                                                  
  Develop core features                     Implement main functionalities like         2023-04-30   Dev Team      
                                            brainstorming tools                                                    
  Integrate with Slack and Trello           Ensure seamless integration with popular    2023-05-31   Dev Team      
                                            tools                                                                  
  Conduct internal testing                  Test the platform internally to identify    2023-06-15   QA Team       
                                            any issues                                                             
  Gather feedback from beta users           Collect and analyze feedback from beta      2023-07-31   QA Team       
                                            testing phase                                                          
  Launch marketing campaigns                Start social media and content marketing    2023-08-01   Marketing     
                                            campaigns                                                              
  Release Free, Pro, and Enterprise tiers   Make different pricing tiers available to   2023-09-01   Sales         
                                            users                                                                  
  Provide customer support                  Set up support channels and address user    2023-10-01   Support Team  
                                            issues                                                                 
  Gather user feedback                      Continuously collect user feedback for      Ongoing      All Teams     
                                            improvements